### Build up model `analysis tables` on regulation-test-2-easy

---

This is a script-like notebook. See [build-analysis-tbls-template.ipynb](./build-analysis-tbls-template.ipynb) for full markdown explanation of what's going on.

    

In [5]:
from fastai2.vision.all import load_learner, get_image_files
import torch
from pathlib import Path
import json, os, sys
import pandas as pd
sys.path.append('..')
from modules.trainutils import piece_class_parse
from modules.trainutils import my_metrics

In [6]:
test_path = Path('../../../other-chess-data/regulation-test-2-easy')
modelconfig_path = Path('../models/model-config')
outtbl_prefix = 'expmod2easy'
out_path = '../models/model-analysis/'
model_dir = '../models/'

# which model .pkl's to search
filter_crit = 'expmod'

### `df_agg'

In [7]:
config_dir = Path(modelconfig_path)

df = pd.DataFrame([])

for fn in os.listdir(config_dir):
    
    with open(config_dir / fn, 'r') as f:
        d = json.load(f)
    
    df = pd.concat((df, pd.DataFrame(d, index=[fn])))

df_agg = df.copy()

In [8]:
df_agg.to_csv(out_path + outtbl_prefix + '-df_agg.csv'
             ,index=True)

### `df_pred'

In [9]:
d_preds = {}
model_fns = [e for e in os.listdir(model_dir) 
             if filter_crit in e]

for model_fn in model_fns:
    
    learn = load_learner(model_dir + model_fn)
    
    test_dl = learn.dls.test_dl(get_image_files(test_path), 
                                with_labels=True)
    
    preds = learn.get_preds(dl= test_dl, with_loss=True)
    
    fns =     [e.name for e in test_dl.items]
    labels =  [piece_class_parse(e) for e in fns]
    
    y_actual = preds[1].tolist()
    y_hat =    torch.argmax(preds[0], dim=1).tolist()
    y_loss =   preds[2].tolist()
    
    mod_name = model_fn.split('.')[0]
    
    d_preds[mod_name]  = {
        'fn':     fns.copy(),
        'label':  labels.copy(),
        'actual': y_actual.copy(),
        'pred':   y_hat.copy(), 
        'loss':   y_loss.copy(),
        }


In [10]:
for i, (mod_name, mod_d) in enumerate(d_preds.items()):
    
    if i == 0:
        
        d_base = {
            'fn':     mod_d['fn'],
            'label':  mod_d['label'],
            'actual': mod_d['actual'],
        }
        
        df = pd.DataFrame(d_base) 
    
    d_learner = {
        ('pred_' + mod_name):   mod_d['pred'], 
        ('loss_' + mod_name):   mod_d['loss'],
    }
    
    df = pd.concat((df, pd.DataFrame(d_learner)), axis=1)
        
df_pred = df.copy()    

In [11]:
df_pred.shape

(162, 135)

In [13]:
df.to_csv('../models/model-analysis/' + outtbl_prefix +'-df_pred.csv',
          index=False)